In [1]:
import numpy as np
import pandas as pd

In [47]:
df = pd.read_csv("../dataset/raw/retail_sales_nigeria.csv")
df.head()

,Order_ID,Order_Date,State,Category,Sub_Category,Segment,Quantity,Sales,Discount,Cost,Profit,Profit_Margin
0,ORD-100000,2023-01-31,Lagos,Furniture,Chairs,Home Office,4,439144.29,0.05,391439.63,47704.66,0.108631
1,ORD-100001,2024-06-26,Oyo,Office Supplies,Paper,Consumer,2,6444.69,0.20,6777.29,-332.61,-0.051610
2,ORD-100002,2023-01-28,Kano,Furniture,Chairs,Consumer,3,76446.25,0.20,78644.94,-2198.69,-0.028761
3,ORD-100003,2022-03-16,Lagos,Furniture,Tables,Consumer,2,92728.49,0.00,83609.53,9118.96,0.098340
4,ORD-100004,2020-01-22,Oyo,Technology,Phones,Home Office,5,1247073.82,0.05,927187.08,319886.74,0.256510


#### DATA QUALITY CHECKS

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Order_ID       200000 non-null  object 
 1   Order_Date     200000 non-null  object 
 2   State          200000 non-null  object 
 3   Category       200000 non-null  object 
 4   Sub_Category   200000 non-null  object 
 5   Segment        200000 non-null  object 
 6   Quantity       200000 non-null  int64  
 7   Sales          200000 non-null  float64
 8   Discount       200000 non-null  float64
 9   Cost           200000 non-null  float64
 10  Profit         200000 non-null  float64
 11  Profit_Margin  200000 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 18.3+ MB


In [49]:
df.isnull().sum()

Order_ID         0
Order_Date       0
State            0
Category         0
Sub_Category     0
Segment          0
Quantity         0
Sales            0
Discount         0
Cost             0
Profit           0
Profit_Margin    0
dtype: int64

In [50]:
df.duplicated().sum()

0

In [51]:
for col in df.columns:
    print(f"{col}:\n{df[col].unique()}\n")


Order_ID:
['ORD-100000' 'ORD-100001' 'ORD-100002' ... 'ORD-299997' 'ORD-299998'
 'ORD-299999']

Order_Date:
['2023-01-31' '2024-06-26' '2023-01-28' ... '2024-10-27' '2024-11-01'
 '2023-08-21']

State:
['Lagos' 'Oyo' 'Kano' 'Plateau' 'Enugu' 'Kaduna' 'Abuja' 'Ogun' 'Anambra'
 'Rivers']

Category:
['Furniture' 'Office Supplies' 'Technology']

Sub_Category:
['Chairs' 'Paper' 'Tables' 'Phones' 'Shelves' 'Laptops' 'Binders'
 'Accessories' 'Storage']

Segment:
['Home Office' 'Consumer' 'Corporate']

Quantity:
[4 2 3 5 1]

Sales:
[439144.29   6444.69  76446.25 ...  29079.32 115519.07   4801.49]

Discount:
[0.05 0.2  0.   0.1  0.35]

Cost:
[391439.63   6777.29  78644.94 ...  21583.1  270903.68   4110.11]

Profit:
[  47704.66    -332.61   -2198.69 ...    7496.22 -155384.61     691.38]

Profit_Margin:
[ 0.10863095 -0.05160993 -0.02876125 ...  0.25778526 -1.34509921
  0.1439928 ]



In [63]:
def wrangle(filepath):
    # Read data file
    df = pd.read_csv("../dataset/raw/retail_sales_nigeria.csv")

    # Convert Order_Date data type
    df["Order_Date"] = pd.to_datetime(df["Order_Date"])

    # Approximate Profit_Margin to two decimal places
    df["Profit_Margin"] = round(df["Profit_Margin"], 2)
    
    # Create Year, Month, Month Name, and Quarter columns based on Date column
    df["Year"] = df["Order_Date"].dt.year
    df["Quarter"] = df["Order_Date"].dt.to_period("Q").astype(str)
    df["Month"] = df["Order_Date"].dt.month
    df["Month_Name"] = df["Order_Date"].dt.month_name()

    # Capture COVID-19 era shocks
    df["COVID_Period"] = np.where(
        (df["Order_Date"] >= "2020-04-01") & (df["Order_Date"] <= "2021-06-30"),
        1,
        0
    )

    # Create Revenue_per_Order column
    df["Revenue_per_Order"] = df["Sales"] / df["Quantity"]

    # Categorize Discounts
    df["Discount_Band"] = pd.cut(
        df["Discount"],
        bins=[-0.01, 0.05, 0.15, 0.30, 1.0],
        labels=["No/Low", "Moderate", "High", "Very High"]
    )

    # Categorize Profit_Margin
    df["Margin_Category"] = pd.cut(
        df["Profit_Margin"],
        bins=[-1, 0.1, 0.25, 1],
        labels=["Low", "Medium", "High"]
    )

    # Segment Customers
    customer_value = (
        df.groupby("Order_ID")
          .agg(
              Total_Sales=("Sales", "sum"),
              Order_Count=("Order_ID", "nunique")
          )
          .reset_index()
    )

    df = df.merge(customer_value, on="Order_ID", how="left")


    return df


In [64]:
df = wrangle("../dataset/raw/retail_sales_nigeria.csv")

In [65]:
df[[
    "Sales", "Profit", "Profit_Margin",
    "Discount", "Discount_Band",
    "Margin_Category", "COVID_Period"
]].head()


,Sales,Profit,Profit_Margin,Discount,Discount_Band,Margin_Category,COVID_Period
0,439144.29,47704.66,0.11,0.05,No/Low,Medium,0
1,6444.69,-332.61,-0.05,0.20,High,Low,0
2,76446.25,-2198.69,-0.03,0.20,High,Low,0
3,92728.49,9118.96,0.10,0.00,No/Low,Low,0
4,1247073.82,319886.74,0.26,0.05,No/Low,High,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Order_ID       200000 non-null  object        
 1   Order_Date     200000 non-null  datetime64[ns]
 2   State          200000 non-null  object        
 3   Category       200000 non-null  object        
 4   Sub_Category   200000 non-null  object        
 5   Segment        200000 non-null  object        
 6   Quantity       200000 non-null  int64         
 7   Sales          200000 non-null  float64       
 8   Discount       200000 non-null  float64       
 9   Cost           200000 non-null  float64       
 10  Profit         200000 non-null  float64       
 11  Profit_Margin  200000 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(5)
memory usage: 18.3+ MB


In [17]:
df["Cost"].unique()

array([391439.63,   6777.29,  78644.94, ...,   9807.81, 392994.69,
         5653.49])

In [19]:
df.duplicated().sum()

0

In [21]:
df.isnull().sum()

Order_ID         0
Order_Date       0
State            0
Category         0
Sub_Category     0
Segment          0
Quantity         0
Sales            0
Discount         0
Cost             0
Profit           0
Profit_Margin    0
dtype: int64

In [27]:
df.to_csv("../dataset/raw/retail_sales_nigeria.csv", index=False)

In [39]:
import numpy as np
import pandas as pd
import random

np.random.seed(42)
random.seed(42)


In [40]:
states = [
    "Lagos", "Abuja", "Rivers", "Oyo", "Ogun",
    "Anambra", "Enugu", "Kaduna", "Kano", "Plateau"
]

categories = {
    "Technology": ["Phones", "Laptops", "Accessories"],
    "Office Supplies": ["Paper", "Binders", "Storage"],
    "Furniture": ["Chairs", "Tables", "Shelves"]
}

segments = ["Consumer", "Corporate", "Home Office"]


In [41]:
dates = pd.date_range(
    start="2020-01-01",
    end="2024-12-31",
    freq="D"
)


In [42]:
def covid_multiplier(order_date, category):
    # Pre-COVID
    if order_date < pd.Timestamp("2020-04-01"):
        return 1.0

    # Lockdown period (Q2–Q3 2020)
    if pd.Timestamp("2020-04-01") <= order_date <= pd.Timestamp("2020-09-30"):
        if category == "Furniture":
            return np.random.uniform(0.4, 0.6)
        elif category == "Technology":
            return np.random.uniform(0.7, 0.9)
        else:  # Office Supplies
            return np.random.uniform(0.8, 1.0)

    # Recovery phase
    if pd.Timestamp("2020-10-01") <= order_date <= pd.Timestamp("2021-06-30"):
        return np.random.uniform(0.85, 1.05)

    # Post-COVID stabilization
    return 1.0


In [43]:
n_orders = 200_000
data = []

for i in range(n_orders):
    order_date = np.random.choice(dates)
    category = random.choice(list(categories.keys()))
    sub_category = random.choice(categories[category])
    state = random.choice(states)
    segment = random.choice(segments)
    
    quantity = np.random.randint(1, 6)

    # Category pricing & margin logic
    if category == "Technology":
        base_price = np.random.uniform(120_000, 450_000)
        margin = np.random.uniform(0.25, 0.40)
    elif category == "Office Supplies":
        base_price = np.random.uniform(2_000, 15_000)
        margin = np.random.uniform(0.15, 0.30)
    else:  # Furniture
        base_price = np.random.uniform(30_000, 120_000)
        margin = np.random.uniform(0.08, 0.18)

    # Discount behavior (COVID-aware)
    if pd.Timestamp("2020-04-01") <= order_date <= pd.Timestamp("2020-09-30"):
        discount = np.random.choice(
            [0.05, 0.10, 0.20, 0.35],
            p=[0.25, 0.35, 0.25, 0.15]
        )
    else:
        discount = np.random.choice(
            [0, 0.05, 0.10, 0.20, 0.35],
            p=[0.45, 0.20, 0.15, 0.15, 0.05]
        )

    # COVID demand shock
    demand_factor = covid_multiplier(order_date, category)

    # Financials
    sales = base_price * quantity * (1 - discount) * demand_factor
    cost = base_price * quantity * (1 - margin)
    profit = sales - cost

    data.append([
        f"ORD-{100000 + i}",
        order_date,
        state,
        category,
        sub_category,
        segment,
        quantity,
        round(sales, 2),
        discount,
        round(cost, 2),
        round(profit, 2)
    ])


In [44]:
columns = [
    "Order_ID", "Order_Date", "State",
    "Category", "Sub_Category", "Segment",
    "Quantity", "Sales", "Discount",
    "Cost", "Profit"
]

df = pd.DataFrame(data, columns=columns)
df["Profit_Margin"] = df["Profit"] / df["Sales"]


In [46]:
df.to_csv("../dataset/raw/retail_sales_nigeria.csv", index=False)

df.head()


,Order_ID,Order_Date,State,Category,Sub_Category,Segment,Quantity,Sales,Discount,Cost,Profit,Profit_Margin
0,ORD-100000,2023-01-31,Lagos,Furniture,Chairs,Home Office,4,439144.29,0.05,391439.63,47704.66,0.108631
1,ORD-100001,2024-06-26,Oyo,Office Supplies,Paper,Consumer,2,6444.69,0.20,6777.29,-332.61,-0.051610
2,ORD-100002,2023-01-28,Kano,Furniture,Chairs,Consumer,3,76446.25,0.20,78644.94,-2198.69,-0.028761
3,ORD-100003,2022-03-16,Lagos,Furniture,Tables,Consumer,2,92728.49,0.00,83609.53,9118.96,0.098340
4,ORD-100004,2020-01-22,Oyo,Technology,Phones,Home Office,5,1247073.82,0.05,927187.08,319886.74,0.256510
